# Какво е представлява производната?
Производна на функцията $f(x)$ може да бъде дефинирана по следния начин:

$$
f'(x) = \lim _{\Delta x \to 0} \frac {f(x + \Delta x) - f(x)} {\Delta x}
$$

### Приближения на производна
Тази дефиниция не може да се реализира в компютърната аритметика, тъй като паметта на всеки компютър е ограничена и това налага определени лимитации от гледна точка на броя значещи цифри след десетичната запетая, с които се представя дадено реално число. Следователно ние можем да направим следното приближение:

$$
f'(x) \approx \frac {f(x + \Delta x) - f(x)} {\Delta x}
$$

Ако формално положим $\Delta x = - \Delta y$, то ще получим друго подобно приближение:
$$
f'(x) \approx \frac {f(x + \Delta x) - f(x)} {\Delta x} = \frac {f(x - \Delta y) - f(x)} {- \Delta y} = \frac {f(x) - f(x - \Delta y)} {\Delta y}
$$
$$
\implies f'(x) \approx \frac {f(x) - f(x - \Delta x)} {\Delta x}
$$

Ако съберем двата израза ще получим:
$$
f'(x) \approx \frac {f(x + \Delta x) - f(x - \Delta x)} {2 \Delta x}
$$

### Визуализация на методите за приближение на производна
В следващите няколко графики може да видите разликите в точността и да сравните грешката, която се допуска в следствие на направеното приближение, в зависимост от използвания метод и стойността на $\Delta x$. Ще работим с функцията $f(x) = e^x$, чиято производна има същия вид - $f'(x) = e^x$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
points = np.arange(-5, 5, 0.01)

def func(num: int) -> float:
    return np.exp(num)

def approximated_derivation_1(num: float, delta: float) -> float:
    return (np.exp(num + delta) - np.exp(num)) / delta

def approximated_derivation_2(num: float, delta: float) -> float:
    return (np.exp(num) - np.exp(num - delta)) / delta

def approximated_derivation_3(num: float, delta: float) -> float:
    return (np.exp(num + delta) - np.exp(num - delta)) / (2 * delta)

all_deltas = [0.5, 10 ** -3, 10 ** -15]
all_derivations = [approximated_derivation_1, approximated_derivation_2, approximated_derivation_3]

for i in range(len(all_derivations)):
    current_derivation = all_derivations[i]

    for j in range(len(all_deltas)):
        current_delta = all_deltas[j]

        plt.title(f'Approximated derivation #{i + 1} with delta {current_delta}')
        plt.xlabel('x')
        plt.ylabel("f'(x)")
        plt.plot(points, [func(p) for p in points], points, [current_derivation(p, current_delta) for p in points], 'r')
        plt.show()

### Анализиране на резултатите

От дефиницията за производна следва, че при намаляване на $\Delta x$ приближението, което получаваме, трябва да бъде по-добро (от дефиницията имаме $\Delta x \to 0$). И наистина - намаляването на грешката се вижда ясно в първите две графики за всеки един от методите на приближение на производната, които сме използвали. При $\Delta x = 10^{-15}$ обаче получената апроксимация не е добра.

Този експеримент нагледно показва това, което бе описано по-рано - тъй като реалните числа в паметта на компютъра се представят с ограничен брой значещи цифри, грешката първоначално ще намалява при намаляване на стъпката, но когато бъде достигнат максимума на машинната точност (и следователно намаляването на стъпката няма повече да подобрява точността на същата тази апроксимация), грешката ще започне да расте заради увеличаването броя на операциите. Това е причината за появата на "шума" на третата графика за всеки един от методите на приближение на производната, които сме използвали.

# Какво представлява численото диференциране?
Численото диференциране ни помага да решаваме практически задачи от този вид: _"Да се намери производна на функцията $f$ в определена точка, като стойностите на функцията са известни само в краен брой точки."_ Също така техниките на численото диференциране са приложими и за функции със сложен аналитичен характер, за които не е оправдано точното пресмятане на производна.

Обикновено решението на подобни задачи представлява намирането на производна на някоя приближаваща функция (като за целта могат да бъдат използвани и интерполационните полиноми. Това, което трябва да се вземе предвид обаче, е, че дори малки изменения на приближаващата функция могат да доведат до по-големи изменения в производната. Нека разгледаме конкретен пример:

$$
f(x) - P(x) = -10^{-5}\sin(mx)
$$

Тук отклонението на приближаващия полином $P(x)$ спрямо функцията $f(x)$ е $R(x) = -10^{-5}\sin(mx)$. Очевидно е, че $|R(x)| \leq 10^{-5}$. Нека обаче да видим какво ще се случи, ако диференцираме двете части на това уравнение:

$$
f'(x)-P'(x) = -m10^{-5}\cos(mx)
$$

Този път ясно се вижда, че грешката $R'(x) = -m10^{-5}\cos(mx)$ може да бъде произволно голяма в зависимост от избраната стойност за параметъра $m$. Следва да заключим, че задачата за числено диференциране не е устойчива.

### Числено диференциране чрез _Интерполационния полином на Нютон_

Нека въведем следните означения:
$$
\omega_n(x) = (x - x_0)(x - x_1)\dots(x - x_n) \\
f[x_i,x_{i+1},\dots,x_{i+k+1}] = \frac {f[x_{i+1},x_{i+2},\dots,x_{i+k+1}] - f[x_i,x_{i+1},\dots,x_{i+k}]} {x_{i+k+1}-x_{i}} \\
$$

Тогава от формулата за представяне на грешка с разделена разлика получаваме:
$$
f(x) = N_n(f;x) + f[x_0,x_1,\dots,x_n,x]\omega_n(x) \\
f'(x) = N_n'(f;x) + \frac {df[x_0,x_1,\dots,x_n,x]} {dx} \omega_n(x) + f[x_0,x_1,\dots,x_n,x] \omega_n'(x)
$$

От дефиницията за производна знаем, че:
$$
\frac {df[x_0,x_1,\dots,x_n,x]} {dx} = \lim_{\Delta x \to 0} \frac {f[x_0,x_1,\dots,x_n,x+\Delta x] - f[x_0,x_1,\dots,x_n,x]} {\Delta x}
$$

За да продължим с по-нататъшното решение на този израз, е необходимо да отбележим едно свойство на разделените разлики, а именно - симетричността им. Ако имаме пермутацията $\sigma : {0,\dots,n} \rightarrow {0,\dots,n}$, то $f[x_0,x_1,\dots,x_n]=f[x_{\sigma(0)},x_{\sigma(1)},\dots,x_{\sigma(n)}]$. Сега можем да се върнем към предишния израз:
$$
\lim_{\Delta x \to 0} \frac {f[x_0,x_1,\dots,x_n,x+\Delta x] - f[x_0,x_1,\dots,x_n,x]} {\Delta x} = \lim_{\Delta x \to 0} f[x_0,x_1,\dots,x_n,x,x + \Delta x] = f[x_0,x_1,\dots,x_n,x,x]
$$

След като знаем как се диференцират разделени разлики, нека се върнем обратно към изчисляването на производната за $f$:
$$
f'(x) = N_n'(f;x) + f[x_0,x_1,\dots,x_n,x,x] \omega_n(x) + f[x_0,x_1,\dots,x_n,x] \omega_n'(x)
$$

Освен това, ако $f \in C^{n+2}[a,b], a \le x_o < x_1 < \dots < x_n \le b, x \in [a,b]$, то тогава можем да опростим записа, като представим разделената разлика чрез производна на $f$ в дадена точка:
$$
f[x_0,x_1,\dots,x_k] = \frac {f^{(k)}(\xi)} {k!} \\
\implies f'(x) = N_n'(f;x) + \frac {f^{(n+2)}(\xi)} {(n+2)!} \omega_n(x) + \frac {f^{(n+1)}(\eta)} {(n+1)!} \omega_n'(x),
$$
където $\xi$ и $\eta$ са точки от интервала $(a, b)$.

От това уравнение можем да заключим, че грешката при приближаване на $f'(x)$ с $N_n'(f;x)$ е:
$$
R'(x) = \frac {f^{(n+2)}(\xi)} {(n+2)!} \omega_n(x) + \frac {f^{(n+1)}(\eta)} {(n+1)!} \omega_n'(x)
$$

Ако направим следните означение - $M_{n+2} = max_{x \in [a,b]}|f^{(n+2)}(x)|$ и $M_{n+1} = max_{x \in [a,b]}|f^{(n+1)}(x)|$, окончателно получаваме:
$$
|R'(x)| \le \frac {M_{n+2}} {(n + 2)!} |\omega_n(x)| + \frac {M_{n+1}} {(n + 1)!} |\omega_n'(x)|
$$

Нека намерим производната на _Интерполационния полином на Нютон_:
$$
N_n(f; x) = f(x_0) + f[x_0, x_1](x - x_0) + f[x_0, x_1, x_2](x - x_0)(x - x_1) + \dots + f[x_0, x_1, \dots, x_n](x - x_0)(x - x_1) \dots (x - x_{n-1}) \\
N_n'(f; x) = f[x_0, x_1] + f[x_0, x_1, x_2][(x - x_0) + (x - x_1)] + f[x_0, x_1, x_2, x_3][(x - x_0)(x - x_1) + (x - x_0)(x - x_2) + (x - x_1)(x - x_2)] + \dots + f[x_0, x_1, \dots, x_n] \sum_{k=0}^{n-1} \prod_{j = 0, j \neq k}^{n-1} (x - x_j)
$$

Този израз е доста комплексен, но лесно може да покажем, че той ще се опрости, когато $x$ приема за стойност някой от интерполационните възли. Нека разгледаме например случая, когато $x = x_0$:
$$
N_n'(f; x) = f[x_0, x_1] + f[x_0, x_1, x_2](x_0 - x_1) + f[x_0, x_1, x_2, x_3](x_0 - x_1)(x_0 - x_2) + \dots + f[x_0, x_1, \dots, x_n](x_0 - x_1)(x_0 - x_2)\dots(x_0 - x_{n-1})
$$

In [ ]:
points_count = int(input("Enter the number of points: "))

x = []
f = []

for i in range(points_count):
    current_point = int(input(f'Enter the x[{i}] value: '))
    current_value = int(input(f'Enter the f(x[{i}]) value: '))
    x.append(current_point)
    f.append(current_value)

divided_differences = [[] for _ in range(points_count)]
for i in range(points_count):
    divided_differences[0].append(f[i])

for k in range(points_count - 1):
    for i in range(len(divided_differences[k]) - 1):
        divided_differences[k + 1].append((divided_differences[k][i + 1] - divided_differences[k][i]) / (x[i + k + 1] - x[i]))

print(divided_differences)
for i in range(points_count):
    for j in range(len(divided_differences[i])):
        divided_difference_notation = ", ".join([f'x[{p}]' for p in range(j, i + j + 1)])
        print(f'f[{divided_difference_notation}] = {divided_differences[i][j]}')
